# Mentoría 'de cómo clasificar en géneros a las canciones'
## Práctico III : Introducción al aprendizaje automático

### Librerías

In [3]:
# ESTA CELDA SOLO PARA COLAB
!pip3 install spotipy
#!pip3 install pandas

# Agregar las librerías extra que se utilicen en esta celda y la siguiente

### Dependencias y acceso al API

In [4]:
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_id = '46b333d567314a89a6254b6c6b054be6'
client_secret = '9d922c3613e441518349dcf55f7d5853'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


### 1) Recopilar los datos obtenidos en los prácticos anteriores

Para esta parte consideraremos [la playlist colaborativa de la mentoría](https://open.spotify.com/playlist/2IuD0qZb14cji5y52crdsO?si=nfHRPDquQRyotEcXc4tG7Q), de esta obtendremos:
- Las features del audio de las canciones
- Las features textuales de sus letras

Además es necesario aplicar el mismo preprocesamiento que aplicamos en los prácticos anteriores para ambos tipos de features (el preprocesamiento del p1 a las features de audio y el de p2 al de features textuales) y obtener el género de cada canción, que en caso de ser más de uno para una canción el equipo deberá discutir una estrategia para estos casos y comentarla en el informe.

Luego, se separará al dataset resultante en **X** e **y**, donde:
- X es el conjunto de features
- y es la etiqueta, en este caso el género de la canción, que deberá ser codificado en valores del tipo **int**

Por último, se dividirá a estos dos conjuntos en los splits **train** y **test**

**Recomendaciones:**
- Obtener las features por separado y hacer un join de los datasets.
- Prestar atención a la [documentación de sklearn](https://scikit-learn.org/stable/)
- Si usan features categóricas, ENCODEARLAS!

In [5]:
dic_features = sp.audio_features('spotify:track:480dDrqG7LO6qDaphHeXlM')
my_features = list(dic_features[0])+['num_artists', 'track_name', 'genero', 'artist_name']

results = sp.playlist_tracks(playlist_id='spotify:playlist:2IuD0qZb14cji5y52crdsO')
items = results['items']
while results['next']:
    results = sp.next(results)
    items.extend(results['items'])

In [13]:
df = pd.DataFrame(columns=my_features)
i = 0
len_items = len(items)
for item in items:
  print(str(i) + ' de ' + str(len_items))
  track_uri = item['track']['uri']
  track_artists = item['track']['artists'] 
  my_row = sp.audio_features(track_uri)[0]
  my_row['num_artists'] = len(track_artists)
  artist = sp.artist(item['track']['artists'][0]['uri'])
  my_row['track_name'] = item['track']['name']
  my_row['genero'] = artist['genres']
  my_row['artist_name'] = artist['name']
  df = df.append(my_row, ignore_index=True)
  i+=1
df

0 de 1188
1 de 1188
2 de 1188
3 de 1188
4 de 1188
5 de 1188
6 de 1188
7 de 1188
8 de 1188
9 de 1188
10 de 1188
11 de 1188
12 de 1188
13 de 1188
14 de 1188
15 de 1188
16 de 1188
17 de 1188
18 de 1188
19 de 1188
20 de 1188
21 de 1188
22 de 1188
23 de 1188
24 de 1188
25 de 1188
26 de 1188
27 de 1188
28 de 1188
29 de 1188
30 de 1188
31 de 1188
32 de 1188
33 de 1188
34 de 1188
35 de 1188
36 de 1188
37 de 1188
38 de 1188
39 de 1188
40 de 1188
41 de 1188
42 de 1188
43 de 1188
44 de 1188
45 de 1188
46 de 1188
47 de 1188
48 de 1188
49 de 1188
50 de 1188
51 de 1188
52 de 1188
53 de 1188
54 de 1188
55 de 1188
56 de 1188
57 de 1188
58 de 1188
59 de 1188
60 de 1188
61 de 1188
62 de 1188
63 de 1188
64 de 1188
65 de 1188
66 de 1188
67 de 1188
68 de 1188
69 de 1188
70 de 1188
71 de 1188
72 de 1188
73 de 1188
74 de 1188
75 de 1188
76 de 1188
77 de 1188
78 de 1188
79 de 1188
80 de 1188
81 de 1188
82 de 1188
83 de 1188
84 de 1188
85 de 1188
86 de 1188
87 de 1188
88 de 1188
89 de 1188
90 de 1188
91 de 118

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,num_artists,track_name,genero,artist_name
0,0.161,0.482,1,-11.210,0,0.0472,0.4860,0.409000,0.371,0.0926,161.092,audio_features,7j9DYPyCuvSAtPcevpAkzb,spotify:track:7j9DYPyCuvSAtPcevpAkzb,https://api.spotify.com/v1/tracks/7j9DYPyCuvSA...,https://api.spotify.com/v1/audio-analysis/7j9D...,233079,5,1,Desafío,"[art pop, dance pop, deconstructed club, elect...",Arca
1,0.230,0.434,10,-12.402,0,0.0551,0.5700,0.000000,0.115,0.0834,80.793,audio_features,1cwTMSQeMaA9fVKEF1iWeD,spotify:track:1cwTMSQeMaA9fVKEF1iWeD,https://api.spotify.com/v1/tracks/1cwTMSQeMaA9...,https://api.spotify.com/v1/audio-analysis/1cwT...,216905,5,1,Anoche,"[art pop, dance pop, deconstructed club, elect...",Arca
2,0.289,0.280,9,-15.335,1,0.0433,0.8740,0.004430,0.124,0.0391,124.835,audio_features,0aL27vskbMpwsMGUkHm3Zf,spotify:track:0aL27vskbMpwsMGUkHm3Zf,https://api.spotify.com/v1/tracks/0aL27vskbMpw...,https://api.spotify.com/v1/audio-analysis/0aL2...,215773,3,1,Sin Rumbo,"[art pop, dance pop, deconstructed club, elect...",Arca
3,0.499,0.648,7,-5.812,1,0.0331,0.7230,0.000000,0.576,0.4640,88.140,audio_features,2kfSFdq2h0xLXq01em1zc7,spotify:track:2kfSFdq2h0xLXq01em1zc7,https://api.spotify.com/v1/tracks/2kfSFdq2h0xL...,https://api.spotify.com/v1/audio-analysis/2kfS...,219107,4,1,La Gata Bajo la Lluvia,"[bolero, cancion melodica, grupera, latin, lat...",Rocío Dúrcal
4,0.528,0.383,2,-11.170,1,0.0258,0.3760,0.000000,0.110,0.4600,89.089,audio_features,5ySxlyvySBhIEvoO2xx7uT,spotify:track:5ySxlyvySBhIEvoO2xx7uT,https://api.spotify.com/v1/tracks/5ySxlyvySBhI...,https://api.spotify.com/v1/audio-analysis/5ySx...,328320,4,1,Querida,"[cancion melodica, latin, latin pop]",Juan Gabriel
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183,0.714,0.753,7,-9.023,1,0.0309,0.0144,0.000003,0.325,0.9100,139.089,audio_features,3TkZFF0JOcc8QXQyxqLB5J,spotify:track:3TkZFF0JOcc8QXQyxqLB5J,https://api.spotify.com/v1/tracks/3TkZFF0JOcc8...,https://api.spotify.com/v1/audio-analysis/3TkZ...,202333,4,1,El Divorcio,"[argentine rock, cuarteto, cumbia pop, cumbia ...",Rodrigo
1184,0.408,0.828,7,-6.222,1,0.0622,0.7400,0.000000,0.322,0.4680,156.506,audio_features,76DYAASchu5szMWFDuG1iP,spotify:track:76DYAASchu5szMWFDuG1iP,https://api.spotify.com/v1/tracks/76DYAASchu5s...,https://api.spotify.com/v1/audio-analysis/76DY...,330493,4,1,"Tema de ""cabecita""","[argentine rock, cuarteto, cumbia pop, cumbia ...",Rodrigo
1185,0.684,0.802,5,-6.619,0,0.0391,0.2360,0.000012,0.438,0.8390,150.167,audio_features,1o7e4Tadn4o6nPvycwIP0C,spotify:track:1o7e4Tadn4o6nPvycwIP0C,https://api.spotify.com/v1/tracks/1o7e4Tadn4o6...,https://api.spotify.com/v1/audio-analysis/1o7e...,260533,4,1,En Libertad,"[argentine rock, cuarteto, cumbia pop, cumbia ...",Rodrigo
1186,0.541,0.734,4,-7.516,0,0.2240,0.5410,0.000000,0.382,0.7040,160.612,audio_features,5AZwGCyE9tIzFYXpqyrKHm,spotify:track:5AZwGCyE9tIzFYXpqyrKHm,https://api.spotify.com/v1/tracks/5AZwGCyE9tIz...,https://api.spotify.com/v1/audio-analysis/5AZw...,250013,4,1,"Amante Tu, Amante El","[argentine rock, cuarteto, cumbia pop, cumbia ...",Rodrigo


In [14]:
#Hay tracks asociados a mas de un artista, por defecto el dataframe tomó el primer artista que figure en la lista
df[df['num_artists'] > 1]

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,num_artists,track_name,genero,artist_name
83,0.679,0.630,7,-6.207,0,0.0283,0.11100,0.000000,0.0423,0.711,100.087,audio_features,20aVSAxv2YvRHYlB4kk1E5,spotify:track:20aVSAxv2YvRHYlB4kk1E5,https://api.spotify.com/v1/tracks/20aVSAxv2YvR...,https://api.spotify.com/v1/audio-analysis/20aV...,270146,4,3,Y se murió de amor,[cuarteto],La Mona Jimenez
84,0.659,0.870,7,-5.216,1,0.0872,0.29000,0.000000,0.2130,0.633,151.179,audio_features,4g4iKJ37XZrOVSNzEqI1PE,spotify:track:4g4iKJ37XZrOVSNzEqI1PE,https://api.spotify.com/v1/tracks/4g4iKJ37XZrO...,https://api.spotify.com/v1/audio-analysis/4g4i...,184360,4,2,Ese guaso,[cuarteto],La Mona Jimenez
85,0.457,0.773,2,-6.957,1,0.2330,0.45300,0.000000,0.7290,0.402,154.012,audio_features,1ql24CwT4y5pZjAJRhpY5a,spotify:track:1ql24CwT4y5pZjAJRhpY5a,https://api.spotify.com/v1/tracks/1ql24CwT4y5p...,https://api.spotify.com/v1/audio-analysis/1ql2...,272640,4,3,Solo tú,[cuarteto],La Mona Jimenez
86,0.529,0.830,9,-5.835,0,0.0685,0.08300,0.000000,0.9320,0.748,153.208,audio_features,1QZ5uqBDf91CsilGfbdV1H,spotify:track:1QZ5uqBDf91CsilGfbdV1H,https://api.spotify.com/v1/tracks/1QZ5uqBDf91C...,https://api.spotify.com/v1/audio-analysis/1QZ5...,222000,4,3,Mujer y amante,[cuarteto],La Mona Jimenez
91,0.669,0.879,11,-5.968,1,0.0977,0.28100,0.000000,0.1500,0.657,150.179,audio_features,0vV3UvotpuHNC3mmKbUpSS,spotify:track:0vV3UvotpuHNC3mmKbUpSS,https://api.spotify.com/v1/tracks/0vV3UvotpuHN...,https://api.spotify.com/v1/audio-analysis/0vV3...,303253,4,3,Beso a beso,[cuarteto],La Mona Jimenez
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132,0.943,0.621,10,-2.603,0,0.0555,0.10700,0.000112,0.3390,0.963,98.046,audio_features,2d7D5eOzALYwuSJUs7WqEH,spotify:track:2d7D5eOzALYwuSJUs7WqEH,https://api.spotify.com/v1/tracks/2d7D5eOzALYw...,https://api.spotify.com/v1/audio-analysis/2d7D...,199560,4,3,Vete de Mi,[cumbia villera],Damas Gratis
1134,0.864,0.813,7,-4.484,1,0.0620,0.24600,0.001810,0.0330,0.794,98.020,audio_features,3RxFQSlNXtcH3gVlAsCRhE,spotify:track:3RxFQSlNXtcH3gVlAsCRhE,https://api.spotify.com/v1/tracks/3RxFQSlNXtcH...,https://api.spotify.com/v1/audio-analysis/3RxF...,1047973,4,10,Batalla Villera,[cumbia villera],Eh!!! Guacho
1135,0.723,0.833,9,-4.677,0,0.0334,0.39600,0.000007,0.1760,0.925,95.008,audio_features,1WKMdjbER963jNYHB3Jrfg,spotify:track:1WKMdjbER963jNYHB3Jrfg,https://api.spotify.com/v1/tracks/1WKMdjbER963...,https://api.spotify.com/v1/audio-analysis/1WKM...,208067,4,11,Amores Como el Nuestro,"[chamame, cumbia villera]",Daniel Cardozo
1137,0.530,0.918,0,-4.486,1,0.0575,0.00102,0.000128,0.6890,0.483,149.504,audio_features,1e4PRJWfSorO7RLgqDRvYR,spotify:track:1e4PRJWfSorO7RLgqDRvYR,https://api.spotify.com/v1/tracks/1e4PRJWfSorO...,https://api.spotify.com/v1/audio-analysis/1e4P...,257808,4,2,Me Extrañarás,"[argentine rock, cuarteto, cumbia pop, cumbia ...",Rodrigo


In [15]:
#Borramos features que no se utilizaran
df.drop(columns=['track_href', 'analysis_url', 'type'], inplace=True)

In [16]:
#Se pasa el feature "mode" a int
df['mode'] = df['mode'].astype(int)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1188 entries, 0 to 1187
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      1188 non-null   float64
 1   energy            1188 non-null   float64
 2   key               1188 non-null   object 
 3   loudness          1188 non-null   float64
 4   mode              1188 non-null   int64  
 5   speechiness       1188 non-null   float64
 6   acousticness      1188 non-null   float64
 7   instrumentalness  1188 non-null   float64
 8   liveness          1188 non-null   float64
 9   valence           1188 non-null   float64
 10  tempo             1188 non-null   float64
 11  id                1188 non-null   object 
 12  uri               1188 non-null   object 
 13  duration_ms       1188 non-null   object 
 14  time_signature    1188 non-null   object 
 15  num_artists       1188 non-null   object 
 16  track_name        1188 non-null   object 


In [18]:
df.describe()

,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
count,1188.000000,1188.000000,1188.000000,1188.000000,1188.000000,1188.000000,1188.000000,1188.000000,1188.000000,1188.000000
mean,0.584945,0.719107,-6.441112,0.608586,0.069333,0.221064,0.019832,0.245267,0.603119,124.744035
std,0.152793,0.182300,2.780900,0.488272,0.066591,0.233256,0.104347,0.235297,0.228579,30.074290
min,0.148000,0.108000,-19.575000,0.000000,0.022600,0.000002,0.000000,0.027700,0.037700,62.850000
25%,0.488000,0.604000,-7.732000,0.000000,0.032475,0.033975,0.000000,0.096650,0.420750,97.031750
50%,0.602000,0.751000,-6.033500,1.000000,0.043600,0.137000,0.000002,0.142000,0.620500,125.002500
75%,0.696250,0.866000,-4.540000,1.000000,0.075050,0.341000,0.000225,0.307250,0.794000,147.262750
max,0.945000,0.995000,-0.767000,1.000000,0.514000,0.982000,0.944000,0.991000,0.976000,204.498000


In [19]:
df.to_csv('practico3_audio_features.csv', index=False)

In [40]:
import requests
import re

In [42]:
re.findall( "^[^\/]*","hola como va / as")

['hola como va ']

In [54]:
df['lyrics']= df.apply(lambda song : (requests.get('https://api.lyrics.ovh/v1/{:}/{:}'.format(song.artist_name, re.findall( "^[^\/]*",song.track_name)[0])).json()), 
         axis=1)

In [55]:
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,uri,duration_ms,time_signature,num_artists,track_name,genero,artist_name,lyrics
0,0.161,0.482,1,-11.210,0,0.0472,0.4860,0.409000,0.371,0.0926,161.092,7j9DYPyCuvSAtPcevpAkzb,spotify:track:7j9DYPyCuvSAtPcevpAkzb,233079,5,1,Desafío,"[art pop, dance pop, deconstructed club, elect...",Arca,{'lyrics': 'Tocame de primera vez Matame una y...
1,0.230,0.434,10,-12.402,0,0.0551,0.5700,0.000000,0.115,0.0834,80.793,1cwTMSQeMaA9fVKEF1iWeD,spotify:track:1cwTMSQeMaA9fVKEF1iWeD,216905,5,1,Anoche,"[art pop, dance pop, deconstructed club, elect...",Arca,{'lyrics': 'Anoche te soñé Tu figura y tus bra...
2,0.289,0.280,9,-15.335,1,0.0433,0.8740,0.004430,0.124,0.0391,124.835,0aL27vskbMpwsMGUkHm3Zf,spotify:track:0aL27vskbMpwsMGUkHm3Zf,215773,3,1,Sin Rumbo,"[art pop, dance pop, deconstructed club, elect...",Arca,{'lyrics': 'Girando en torno al sol Te pierdo ...
3,0.499,0.648,7,-5.812,1,0.0331,0.7230,0.000000,0.576,0.4640,88.140,2kfSFdq2h0xLXq01em1zc7,spotify:track:2kfSFdq2h0xLXq01em1zc7,219107,4,1,La Gata Bajo la Lluvia,"[bolero, cancion melodica, grupera, latin, lat...",Rocío Dúrcal,{'lyrics': 'Amor tranquilo no te voy a molesta...
4,0.528,0.383,2,-11.170,1,0.0258,0.3760,0.000000,0.110,0.4600,89.089,5ySxlyvySBhIEvoO2xx7uT,spotify:track:5ySxlyvySBhIEvoO2xx7uT,328320,4,1,Querida,"[cancion melodica, latin, latin pop]",Juan Gabriel,"{'lyrics': 'Querida, cada momento de mi vida y..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183,0.714,0.753,7,-9.023,1,0.0309,0.0144,0.000003,0.325,0.9100,139.089,3TkZFF0JOcc8QXQyxqLB5J,spotify:track:3TkZFF0JOcc8QXQyxqLB5J,202333,4,1,El Divorcio,"[argentine rock, cuarteto, cumbia pop, cumbia ...",Rodrigo,{'error': 'No lyrics found'}
1184,0.408,0.828,7,-6.222,1,0.0622,0.7400,0.000000,0.322,0.4680,156.506,76DYAASchu5szMWFDuG1iP,spotify:track:76DYAASchu5szMWFDuG1iP,330493,4,1,"Tema de ""cabecita""","[argentine rock, cuarteto, cumbia pop, cumbia ...",Rodrigo,{'error': 'No lyrics found'}
1185,0.684,0.802,5,-6.619,0,0.0391,0.2360,0.000012,0.438,0.8390,150.167,1o7e4Tadn4o6nPvycwIP0C,spotify:track:1o7e4Tadn4o6nPvycwIP0C,260533,4,1,En Libertad,"[argentine rock, cuarteto, cumbia pop, cumbia ...",Rodrigo,{'error': 'No lyrics found'}
1186,0.541,0.734,4,-7.516,0,0.2240,0.5410,0.000000,0.382,0.7040,160.612,5AZwGCyE9tIzFYXpqyrKHm,spotify:track:5AZwGCyE9tIzFYXpqyrKHm,250013,4,1,"Amante Tu, Amante El","[argentine rock, cuarteto, cumbia pop, cumbia ...",Rodrigo,{'error': 'No lyrics found'}


In [56]:
df.to_csv('practico3_audio_features.csv', index=False)

In [57]:
df_test = pd.read_csv('practico3_audio_features.csv')

In [58]:
df_test

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,uri,duration_ms,time_signature,num_artists,track_name,genero,artist_name,lyrics
0,0.161,0.482,1,-11.210,0,0.0472,0.4860,0.409000,0.371,0.0926,161.092,7j9DYPyCuvSAtPcevpAkzb,spotify:track:7j9DYPyCuvSAtPcevpAkzb,233079,5,1,Desafío,"['art pop', 'dance pop', 'deconstructed club',...",Arca,{'lyrics': 'Tocame de primera vez\nMatame una ...
1,0.230,0.434,10,-12.402,0,0.0551,0.5700,0.000000,0.115,0.0834,80.793,1cwTMSQeMaA9fVKEF1iWeD,spotify:track:1cwTMSQeMaA9fVKEF1iWeD,216905,5,1,Anoche,"['art pop', 'dance pop', 'deconstructed club',...",Arca,{'lyrics': 'Anoche te soñé\nTu figura y tus br...
2,0.289,0.280,9,-15.335,1,0.0433,0.8740,0.004430,0.124,0.0391,124.835,0aL27vskbMpwsMGUkHm3Zf,spotify:track:0aL27vskbMpwsMGUkHm3Zf,215773,3,1,Sin Rumbo,"['art pop', 'dance pop', 'deconstructed club',...",Arca,{'lyrics': 'Girando en torno al sol\nTe pierdo...
3,0.499,0.648,7,-5.812,1,0.0331,0.7230,0.000000,0.576,0.4640,88.140,2kfSFdq2h0xLXq01em1zc7,spotify:track:2kfSFdq2h0xLXq01em1zc7,219107,4,1,La Gata Bajo la Lluvia,"['bolero', 'cancion melodica', 'grupera', 'lat...",Rocío Dúrcal,{'lyrics': 'Amor tranquilo no te voy a molesta...
4,0.528,0.383,2,-11.170,1,0.0258,0.3760,0.000000,0.110,0.4600,89.089,5ySxlyvySBhIEvoO2xx7uT,spotify:track:5ySxlyvySBhIEvoO2xx7uT,328320,4,1,Querida,"['cancion melodica', 'latin', 'latin pop']",Juan Gabriel,"{'lyrics': 'Querida,\ncada momento de mi vida\..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1183,0.714,0.753,7,-9.023,1,0.0309,0.0144,0.000003,0.325,0.9100,139.089,3TkZFF0JOcc8QXQyxqLB5J,spotify:track:3TkZFF0JOcc8QXQyxqLB5J,202333,4,1,El Divorcio,"['argentine rock', 'cuarteto', 'cumbia pop', '...",Rodrigo,{'error': 'No lyrics found'}
1184,0.408,0.828,7,-6.222,1,0.0622,0.7400,0.000000,0.322,0.4680,156.506,76DYAASchu5szMWFDuG1iP,spotify:track:76DYAASchu5szMWFDuG1iP,330493,4,1,"Tema de ""cabecita""","['argentine rock', 'cuarteto', 'cumbia pop', '...",Rodrigo,{'error': 'No lyrics found'}
1185,0.684,0.802,5,-6.619,0,0.0391,0.2360,0.000012,0.438,0.8390,150.167,1o7e4Tadn4o6nPvycwIP0C,spotify:track:1o7e4Tadn4o6nPvycwIP0C,260533,4,1,En Libertad,"['argentine rock', 'cuarteto', 'cumbia pop', '...",Rodrigo,{'error': 'No lyrics found'}
1186,0.541,0.734,4,-7.516,0,0.2240,0.5410,0.000000,0.382,0.7040,160.612,5AZwGCyE9tIzFYXpqyrKHm,spotify:track:5AZwGCyE9tIzFYXpqyrKHm,250013,4,1,"Amante Tu, Amante El","['argentine rock', 'cuarteto', 'cumbia pop', '...",Rodrigo,{'error': 'No lyrics found'}
